In [30]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_score, mean_absolute_error
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn import svm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
import sklearn

np.set_printoptions(precision=4)

In [31]:
data = pd.read_csv('final_df.csv')

display(data)

,season,round,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy,driver,grid,podium,...,constructor_minardi,constructor_prost,constructor_red_bull,constructor_renault,constructor_sauber,constructor_team_lotus,constructor_toro_rosso,constructor_toyota,constructor_tyrrell,constructor_williams
0,1983,1,False,False,True,False,False,keke_rosberg,1,15,...,0,0,0,0,0,0,0,0,0,1
1,1983,1,False,False,True,False,False,prost,2,6,...,0,0,0,1,0,0,0,0,0,0
2,1983,1,False,False,True,False,False,tambay,3,4,...,0,0,0,0,0,0,0,0,0,0
3,1983,1,False,False,True,False,False,piquet,4,1,...,0,0,0,0,0,0,0,0,0,0
4,1983,1,False,False,True,False,False,warwick,5,7,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14268,2019,21,True,False,False,False,False,giovinazzi,16,16,...,0,0,0,0,0,0,0,0,0,0
14269,2019,21,True,False,False,False,False,raikkonen,17,13,...,0,0,0,0,0,0,0,0,0,0
14270,2019,21,True,False,False,False,False,russell,18,17,...,0,0,0,0,0,0,0,0,0,1
14271,2019,21,True,False,False,False,False,kubica,19,19,...,0,0,0,0,0,0,0,0,0,1


In [32]:
df = data.copy()

train = df[df.season <2019]
X_train = train.drop(['driver', 'podium'], axis = 1)
y_train = train.podium

scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)

In [33]:
comparison_dict ={'model':[],
                  'params': [],
                  'score': []}

# Implement Model

In [34]:
model = sklearn.linear_model.LogisticRegression()
model.fit(X_train, y_train)

/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

# Evaluate Model

In [35]:
def score_classification(model):
    score = 0
    for circuit in df[df.season == 2019]['round'].unique():

        test = df[(df.season == 2019) & (df['round'] == circuit)]
        X_test = test.drop(['driver', 'podium'], axis = 1)
        y_test = test.podium

        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)

        # make predictions
        #test = np.transpose(np.array(model.predict(X_test, return_std=True)))
        test = model.predict(X_test)
        #prediction_df = pd.DataFrame(test, columns = ['y_mean', 'y_std'])
        prediction_df = pd.DataFrame(test, columns=['y_pred'])
        prediction_df['actual'] = y_test.reset_index(drop = True)

        #prediction_df = prediction_df[prediction_df.y_std < 2]

        #score += mean_absolute_error(prediction_df.actual, prediction_df.y_mean)
        score += mean_absolute_error(prediction_df.actual, prediction_df.y_pred)

    model_score = score / df[df.season == 2019]['round'].unique().max()
    return model_score

In [36]:
model_score = score_classification(model)

comparison_dict['model'].append('gpr')
comparison_dict['score'].append(model_score)

display(comparison_dict)

{'model': ['gpr'], 'params': [], 'score': [4.051886661097187]}

In [37]:
circuits = df[df.season == 2019]['round'].unique()
test_circuit = df[(df.season == 2019) & (df['round'] == circuits[0])]

X_test = test_circuit.drop(['driver', 'podium'], axis = 1)
y_test = test_circuit.podium

#scaling
X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)

# make predictions
#test = np.transpose(np.array(model.predict(X_test, return_std=True)))
test = pd.DataFrame(model.predict_proba(X_test))

display(X_test)
display(test)


,season,round,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy,grid,driver_points,driver_wins,...,constructor_minardi,constructor_prost,constructor_red_bull,constructor_renault,constructor_sauber,constructor_team_lotus,constructor_toro_rosso,constructor_toyota,constructor_tyrrell,constructor_williams
0,1.760896,-1.609257,1.162719,-0.153859,-0.552022,-0.338102,-0.362729,-1.609286,-0.465586,-0.310681,...,-0.216773,-0.109343,-0.194304,-0.203503,-0.239664,-0.163869,-0.185550,-0.140043,-0.187617,-0.311532
1,1.760896,-1.609257,1.162719,-0.153859,-0.552022,-0.338102,-0.362729,-1.461131,-0.465586,-0.310681,...,-0.216773,-0.109343,-0.194304,-0.203503,-0.239664,-0.163869,-0.185550,-0.140043,-0.187617,-0.311532
2,1.760896,-1.609257,1.162719,-0.153859,-0.552022,-0.338102,-0.362729,-1.312976,-0.465586,-0.310681,...,-0.216773,-0.109343,-0.194304,-0.203503,-0.239664,-0.163869,-0.185550,-0.140043,-0.187617,-0.311532
3,1.760896,-1.609257,1.162719,-0.153859,-0.552022,-0.338102,-0.362729,-1.164821,-0.465586,-0.310681,...,-0.216773,-0.109343,5.146565,-0.203503,-0.239664,-0.163869,-0.185550,-0.140043,-0.187617,-0.311532
4,1.760896,-1.609257,1.162719,-0.153859,-0.552022,-0.338102,-0.362729,-1.016666,-0.465586,-0.310681,...,-0.216773,-0.109343,-0.194304,-0.203503,-0.239664,-0.163869,-0.185550,-0.140043,-0.187617,-0.311532
5,1.760896,-1.609257,1.162719,-0.153859,-0.552022,-0.338102,-0.362729,-0.868512,-0.465586,-0.310681,...,-0.216773,-0.109343,-0.194304,-0.203503,-0.239664,-0.163869,-0.185550,-0.140043,-0.187617,-0.311532
6,1.760896,-1.609257,1.162719,-0.153859,-0.552022,-0.338102,-0.362729,-0.720357,-0.465586,-0.310681,...,-0.216773,-0.109343,-0.194304,-0.203503,-0.239664,-0.163869,-0.185550,-0.140043,-0.187617,-0.311532
7,1.760896,-1.609257,1.162719,-0.153859,-0.552022,-0.338102,-0.362729,-0.572202,-0.465586,-0.310681,...,-0.216773,-0.109343,-0.194304,-0.203503,-0.239664,-0.163869,-0.185550,-0.140043,-0.187617,-0.311532
8,1.760896,-1.609257,1.162719,-0.153859,-0.552022,-0.338102,-0.362729,-0.424047,-0.465586,-0.310681,...,-0.216773,-0.109343,-0.194304,-0.203503,-0.239664,-0.163869,-0.185550,-0.140043,-0.187617,-0.311532
9,1.760896,-1.609257,1.162719,-0.153859,-0.552022,-0.338102,-0.362729,-0.275892,-0.465586,-0.310681,...,-0.216773,-0.109343,-0.194304,-0.203503,-0.239664,-0.163869,-0.185550,-0.140043,-0.187617,-0.311532


,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,0.399439,0.125656,0.062115,0.050438,0.069343,0.038450,0.047665,0.009155,0.027414,0.013667,...,0.010505,0.019977,0.004754,0.009967,0.011521,0.001172,0.000634,0.000013,3.536764e-06,0.000001
1,0.174885,0.169632,0.083872,0.068760,0.103727,0.043712,0.056427,0.016340,0.045600,0.020915,...,0.017109,0.027788,0.006675,0.012189,0.008696,0.002019,0.001369,0.000011,1.449795e-07,0.000004
2,0.166973,0.138030,0.129849,0.081250,0.057411,0.061504,0.025966,0.018596,0.019615,0.012807,...,0.042589,0.025298,0.019983,0.022062,0.025003,0.000875,0.000345,0.000010,1.313495e-05,0.000005
3,0.153089,0.171571,0.126610,0.063570,0.081295,0.019263,0.022379,0.033779,0.019424,0.017824,...,0.063960,0.017734,0.023421,0.013870,0.010617,0.000106,0.000404,0.000005,1.704083e-06,0.000007
4,0.033533,0.129635,0.114615,0.060931,0.052445,0.027958,0.032368,0.021037,0.033723,0.024381,...,0.068354,0.040385,0.021059,0.037312,0.020463,0.001382,0.000368,0.000016,1.687110e-05,0.000045
5,0.001122,0.001024,0.000834,0.024147,0.059868,0.089316,0.046877,0.089792,0.041007,0.043272,...,0.025980,0.058028,0.062826,0.017365,0.022141,0.000055,0.000049,0.000016,6.249589e-06,0.000012
6,0.000699,0.001004,0.000602,0.021616,0.051161,0.043197,0.062301,0.085164,0.036559,0.057957,...,0.029684,0.065571,0.047211,0.020462,0.015777,0.000056,0.000031,0.000011,7.128920e-06,0.000012
7,0.012783,0.045234,0.072511,0.047558,0.072095,0.044218,0.050516,0.027563,0.045520,0.028654,...,0.069285,0.072638,0.036369,0.026584,0.020561,0.001225,0.000913,0.000023,1.603015e-05,0.000010
8,0.006224,0.025504,0.033676,0.058948,0.060076,0.052771,0.039797,0.045033,0.032504,0.031680,...,0.094977,0.063924,0.059531,0.048447,0.029416,0.004625,0.001501,0.000048,1.781420e-06,0.000001
9,0.000556,0.065767,0.082856,0.017674,0.039376,0.028737,0.052663,0.029760,0.031648,0.030191,...,0.027994,0.064528,0.047075,0.056928,0.046669,0.000064,0.002148,0.000024,2.150587e-05,0.000010


In [47]:
df_420 = test
df_420['DNF'] = test[20] + test[21] + test[22] + test[23] + test[24] + test[25] + test[26]
df_420.drop([20, 21, 22, 23, 24, 25, 26], axis = 1)

display(df_420)

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,DNF
0,0.399439,0.125656,0.062115,0.050438,0.069343,0.038450,0.047665,0.009155,0.027414,0.013667,...,0.019977,0.004754,0.009967,0.011521,0.001172,0.000634,0.000013,3.536764e-06,0.000001,0.023312
1,0.174885,0.169632,0.083872,0.068760,0.103727,0.043712,0.056427,0.016340,0.045600,0.020915,...,0.027788,0.006675,0.012189,0.008696,0.002019,0.001369,0.000011,1.449795e-07,0.000004,0.024287
2,0.166973,0.138030,0.129849,0.081250,0.057411,0.061504,0.025966,0.018596,0.019615,0.012807,...,0.025298,0.019983,0.022062,0.025003,0.000875,0.000345,0.000010,1.313495e-05,0.000005,0.048315
3,0.153089,0.171571,0.126610,0.063570,0.081295,0.019263,0.022379,0.033779,0.019424,0.017824,...,0.017734,0.023421,0.013870,0.010617,0.000106,0.000404,0.000005,1.704083e-06,0.000007,0.025010
4,0.033533,0.129635,0.114615,0.060931,0.052445,0.027958,0.032368,0.021037,0.033723,0.024381,...,0.040385,0.021059,0.037312,0.020463,0.001382,0.000368,0.000016,1.687110e-05,0.000045,0.059603
5,0.001122,0.001024,0.000834,0.024147,0.059868,0.089316,0.046877,0.089792,0.041007,0.043272,...,0.058028,0.062826,0.017365,0.022141,0.000055,0.000049,0.000016,6.249589e-06,0.000012,0.039644
6,0.000699,0.001004,0.000602,0.021616,0.051161,0.043197,0.062301,0.085164,0.036559,0.057957,...,0.065571,0.047211,0.020462,0.015777,0.000056,0.000031,0.000011,7.128920e-06,0.000012,0.036356
7,0.012783,0.045234,0.072511,0.047558,0.072095,0.044218,0.050516,0.027563,0.045520,0.028654,...,0.072638,0.036369,0.026584,0.020561,0.001225,0.000913,0.000023,1.603015e-05,0.000010,0.049332
8,0.006224,0.025504,0.033676,0.058948,0.060076,0.052771,0.039797,0.045033,0.032504,0.031680,...,0.063924,0.059531,0.048447,0.029416,0.004625,0.001501,0.000048,1.781420e-06,0.000001,0.084040
9,0.000556,0.065767,0.082856,0.017674,0.039376,0.028737,0.052663,0.029760,0.031648,0.030191,...,0.064528,0.047075,0.056928,0.046669,0.000064,0.002148,0.000024,2.150587e-05,0.000010,0.105865
